### Basic imports

In [ ]:
# Basic notebook imports
%matplotlib inline
import os
import matplotlib
import pylab as plt
import numpy as np
import healpy as hp
import easyaccess as ea
import pandas as pd

### We use skymap, a matplotlib based utility to draw celestials maps

In [ ]:
import skymap
from skymap.constants import FIGSIZE,SCALE,DPI,DECAM
from skymap.utils import get_datadir
from skymap import SurveySkymap,SurveyMcBryde,SurveyOrtho
from skymap import DESSkymap

### Read data (stars) and group them by HPIX 1024 (which doesn't exists si we use the degrade functions)

In [ ]:
# Query used to generate the data (it takes ~15 min)
query = """
--
-- Example Query --
-- This query creates a Helpix map of number of starts
-- and their mean magnitude on a resolution of NSIDE = 1024
-- using NEST Schema
SELECT
count(MAG_AUTO_I) COUNT,
MCARRAS2.DEGRADE(HPIX_16384, 1024) AS HPIX_1024
FROM Y3_GOLD_2_2 
WHERE
  WAVG_SPREAD_MODEL_I + 3.0*WAVG_SPREADERR_MODEL_I < 0.005 and
  WAVG_SPREAD_MODEL_I > -1 and
  IMAFLAGS_ISO_I = 0 and
  MAG_AUTO_I < 21
GROUP BY MCARRAS2.DEGRADE(HPIX_16384, 1024)
"""

In [ ]:
con = ea.connect()

In [ ]:
df = con.query_to_pandas(query)

In [ ]:
# Load footprint
raf,decf = np.loadtxt('round17-poly.txt',unpack=True)

In [ ]:
# Convert from NESTED to RING format for the plots
px2 = hp.nest2ring(1024,df.HPIX_1024)

In [ ]:
NSIDE = 1024
plt.figure(figsize=(16,12))
matplotlib.rc('xtick', labelsize=32) 
matplotlib.rc('ytick', labelsize=32) 
matplotlib.rc('axes' , labelsize=32)
smap = DESSkymap()
#smap.draw_des(color='red',lw=3)
smap.draw_hpxmap(df['COUNT'],px2,nside=1024,xsize=1000)
smap.draw_inset_colorbar()
smap.draw_polygon_radec(raf,decf,color='r',lw=3.5)
ax = plt.gca()
ax.axis['right'].major_ticklabels.set_visible(False)